In [1]:
import pandas as pd
import numpy as np

In [2]:
Revenue = pd.read_csv('Revenue_Analysis_Test_Data.csv')
Revenue.head() 

,ACTIVITY_DATE,MEMBER_ID,GAME_ID,WAGER_AMOUNT,NUMBER_OF_WAGERS,WIN_AMOUNT,ACTIVITY_YEAR_MONTH,BANK_TYPE_ID
0,2017-01-20,1001,987,0.1,1,0.0,201701,0
1,2017-01-27,1001,7499,0.1,1,0.7,201701,0
2,2017-01-27,1001,7499,0.2,1,0.0,201701,0
3,2017-01-27,1001,1320,0.2,1,13.4,201701,0
4,2017-01-05,1001,1293,0.5,1,0.0,201701,0


In [3]:
Calendar = pd.read_csv('Calendar_Test_Data.csv')
Calendar.head()

,CALENDAR_DATE,CALENDAR_YEAR,CALENDAR_MONTH_NUMBER,CALENDAR_MONTH_NAME,CALENDAR_DAY_OF_MONTH,CALENDAR_DAY_OF_WEEK,CALENDAR_DAY_NAME,CALENDAR_YEAR_MONTH
0,2015-01-01,2015,1,January,1,5,Thursday,201501
1,2015-01-02,2015,1,January,2,6,Friday,201501
2,2015-01-03,2015,1,January,3,7,Saturday,201501
3,2015-01-04,2015,1,January,4,1,Sunday,201501
4,2015-01-05,2015,1,January,5,2,Monday,201501


In [4]:
member_id = Revenue['MEMBER_ID'].unique().tolist()
member_id[0]

1001

In [5]:
member_id

[1001, 1002, 1003]

In [6]:
df_final = pd.DataFrame()
for i in member_id: 
    df_member = Revenue[['MEMBER_ID','ACTIVITY_YEAR_MONTH','NUMBER_OF_WAGERS']].loc[Revenue['MEMBER_ID'] == i]
    df_member = df_member.groupby(['MEMBER_ID','ACTIVITY_YEAR_MONTH'])['NUMBER_OF_WAGERS'].count()
    df_member = pd.DataFrame(df_member).reset_index()

    year_month_min = min(df_member['ACTIVITY_YEAR_MONTH'])
    year_month_max = max(df_member['ACTIVITY_YEAR_MONTH'])

    df_calendar = pd.DataFrame(Calendar['CALENDAR_YEAR_MONTH'].unique())
    df_calendar = df_calendar.rename(columns={0: 'CALENDAR_YEAR_MONTH'})
    df_calendar = df_calendar.loc[df_calendar['CALENDAR_YEAR_MONTH'].between(year_month_min-1,year_month_max+1, inclusive=False)]
    df_calendar = df_calendar.reset_index(drop=True)

    df_main = pd.merge(df_member, df_calendar, left_on='ACTIVITY_YEAR_MONTH', right_on='CALENDAR_YEAR_MONTH', how='right')
    df_main = df_main.drop('ACTIVITY_YEAR_MONTH', axis=1)
    df_main = df_main[['MEMBER_ID', 'CALENDAR_YEAR_MONTH', 'NUMBER_OF_WAGERS']]
    df_main['MEMBER_ID'] = df_main['MEMBER_ID'].fillna(i)
    df_main['NUMBER_OF_WAGERS'] = df_main['NUMBER_OF_WAGERS'].fillna(0)
    df_main = df_main.astype(int)

    df_lifecycle = pd.DataFrame()
    for i in df_main.index:
        if df_main.index[i] == 0 :
            df_new = df_main.iloc[[i]]
            df_new['MEMBER_LIFECYCLE_STATUS'] = 'new'
        else:
            df_new = df_main.iloc[[i-1,i]]
            if df_new['NUMBER_OF_WAGERS'][i-1] != 0 and df_new['NUMBER_OF_WAGERS'][i] != 0:
                df_new['MEMBER_LIFECYCLE_STATUS'] = 'Retained'
            elif df_new['NUMBER_OF_WAGERS'][i-1] != 0 and df_new['NUMBER_OF_WAGERS'][i] == 0:
                df_new['MEMBER_LIFECYCLE_STATUS'] = 'Unretained'
            elif df_new['NUMBER_OF_WAGERS'][i-1] == 0 and df_new['NUMBER_OF_WAGERS'][i] != 0:
                df_new['MEMBER_LIFECYCLE_STATUS'] = 'Reactivated'
            elif df_new['NUMBER_OF_WAGERS'][i-1] == 0 and df_new['NUMBER_OF_WAGERS'][i] == 0:
                df_new['MEMBER_LIFECYCLE_STATUS'] = 'Lapsed'
            df_new = df_new.drop(i-1)
        df_lifecycle = df_lifecycle.append(df_new)

    df_lifecycle['LAPSED_MONTHS'] = np.where(df_lifecycle['MEMBER_LIFECYCLE_STATUS']=='Lapsed', 
    df_lifecycle.groupby((df_lifecycle['MEMBER_LIFECYCLE_STATUS'] != df_lifecycle['MEMBER_LIFECYCLE_STATUS'].shift(1)).cumsum()).cumcount()+2, 'NA')

    df_final = df_final.append(df_lifecycle)

<ipython-input-6-a17785d3fa4d>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['MEMBER_LIFECYCLE_STATUS'] = 'new'
<ipython-input-6-a17785d3fa4d>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['MEMBER_LIFECYCLE_STATUS'] = 'Unretained'
<ipython-input-6-a17785d3fa4d>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [7]:
df_final = df_final.drop('NUMBER_OF_WAGERS', axis=1)
df_final

,MEMBER_ID,CALENDAR_YEAR_MONTH,MEMBER_LIFECYCLE_STATUS,LAPSED_MONTHS
0,1001,201701,new,NA
1,1001,201702,Unretained,NA
2,1001,201703,Lapsed,2
3,1001,201704,Lapsed,3
4,1001,201705,Reactivated,NA
5,1001,201706,Retained,NA
6,1001,201707,Unretained,NA
7,1001,201708,Lapsed,2
8,1001,201709,Reactivated,NA
9,1001,201710,Unretained,NA


In [8]:
df_final.to_csv('view.csv', index=False)